In [1]:
%matplotlib inline
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras import regularizers
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split

import keras
import matplotlib.pyplot as plt
from rasterio.plot import show
import xarray as xr

import gdal
import rasterio
import glob
import os
import numpy as np
import pandas as pd
import pandas_profiling as pp

Using TensorFlow backend.
/home/datacube/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/datacube/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/datacube/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/datacube/.local/lib/python3.6/site-packages/tensorflo

In [2]:
data = pd.read_csv('./satellite_dataset.csv')
data.head()

,blue,green,red,nir,swir1,swir2,wofs,bosque,wofs_bosque,ninguno
0,203.0,131.0,36.0,21.0,61.0,46.0,1,0,0,0
1,207.0,137.0,39.0,27.0,71.0,59.0,1,0,0,0
2,393.0,325.0,217.0,200.0,254.0,176.0,1,0,0,0
3,269.0,201.0,58.0,11.0,51.0,43.0,1,0,0,0
4,194.0,218.0,112.0,103.0,153.0,130.0,1,0,0,0


In [3]:
# pp.ProfileReport(data)

In [4]:
dataset_X = data.drop(columns=['wofs','bosque','wofs_bosque','ninguno'], axis=1)
dataset_X.head()

,blue,green,red,nir,swir1,swir2
0,203.0,131.0,36.0,21.0,61.0,46.0
1,207.0,137.0,39.0,27.0,71.0,59.0
2,393.0,325.0,217.0,200.0,254.0,176.0
3,269.0,201.0,58.0,11.0,51.0,43.0
4,194.0,218.0,112.0,103.0,153.0,130.0


In [5]:
dataset_Y = data[['wofs','bosque','wofs_bosque','ninguno']]
dataset_Y.head()

,wofs,bosque,wofs_bosque,ninguno
0,1,0,0,0
1,1,0,0,0
2,1,0,0,0
3,1,0,0,0
4,1,0,0,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_Y, test_size=0.2)
X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [7]:
# DEFINICIÓN DEL MODELO
# Hay dos maneras de construir modelos en Keras: Secuencial y Funcional
# El modelo secuencial permite construir modelos capa por capa.
# El modelo funcional permite construir modelos mas complicados.

# La capa Flaten transforma los datos de un arreglo bidimensional de 28x28 a un arreglo 
# unidimensional de 784 posiciones(28x28) esto solo formatea el conjutno de datos

# La capa Dense significa que cada neurona en una capa esta conectada a todas las neuronas 
# localizadas en la capa anterior y con todas en la siguiente capa.

model = keras.Sequential([
#     keras.layers.Flatten(input_shape=(6,)),
    keras.layers.Dense(128,input_dim=6, activation='relu'),
    keras.layers.Dense(4, activation='softmax')
])

# CONFIGURACIÓN DE PARÁMETROS
# La compilación del modelo comprende la configuración de parámetros 
# usados durante el entrenamiento: algori tmo de optimización, medida 
# de exactitud, etc.

# optimizer: Adam es un algoitmo de optimización. Además es un método de tasa de aprendizaje adaptativo, 
# loss: calcula la diferencia entre la salida y la variable objetivo. Mide la precisión del modelo durante 
# el entrenamiento y queremos minimizar esta función.
# metrics: Son las métricas que se desan calcular durante el entrenamiento. mide la fracción de imágenes que 
# están clasificadas correctamente.

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Resumen de parámetros del modelo
model.summary()

# ENTRENAMIENTO
model.fit(X_train, y_train, epochs=15)

# EVALUACIÓN
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               896       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 516       
Total params: 1,412
Trainable params: 1,412
Non-trainable params: 0
_________________________________________________________________

Epoch 1/15
800000/800000 [==============================] - 21s 27us/step - loss: 0.2807 - accuracy: 0.9739
Epoch 2/15
800000/800000 [==============================] - 21s 27us/step - loss: 0.0514 - accuracy: 0.9810
Epoch 3/15
800000/800000 [==============================] - 21s 26us/step - loss: 0.0513 - accuracy: 0.9814
Epoch 4/15
800000/800000 [==============================] - 21s 26us/step - loss: 0.0490 - accuracy: 0.9822
Epoch 5/15
800000/800000 [==============================] - 21s 26us/step - loss:

In [8]:
model.save('my_model.h5')

In [9]:
y_train

array([[1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       ...,
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 0]])

In [10]:
X_train

array([[437., 326., 234., 231., 272., 205.],
       [240., 227., 126., 118., 148., 121.],
       [274., 238., 140., 121., 149., 119.],
       ...,
       [175.,  98.,   3.,  -7.,  47.,  40.],
       [184., 162.,  84.,  80., 124., 102.],
       [269., 174.,  70.,  50.,  81.,  62.]])

In [11]:
def predict_with_class(model,instance=[]):
    proba = model.predict([[ instance ]])
    index = np.argmax(proba)
    # classes = ['agua','bosque','agua_o_bosque','ninguno']
    classes = [1,2,3,4]
    return classes[index]

In [12]:
predict_with_class(model,[267,175,70,52,87,73])

1

In [13]:
path = '../data/prueba/pequena.tif'

def get_bands_positions(path):
    """
    Permite obtener las posiciones y los nombres de las bandas
    a partir de los metadatos de la imágen tif.
    
    [(1, 'green'), (2, 'swir2'), (3, 'nir'), (4, 'blue'), (5, 'red'), (6, 'swir1')]
    """
    ds = gdal.Open(path)
    # Extracción de los metadatos del tif
    metadata = ds.GetMetadata()
    # Extracción de la posición y nombre de las bandas
    positions = [ (int(k[-1]) - 1,v) for k,v in metadata.items() if 'Band_' in k ]
    
    return positions


def load_ideam_geotif(path):
    
    positions = get_bands_positions(path)
    
    data_array = xr.open_rasterio(path)

    # Define las coordenadas de los puntos
    coords = { 
        'x':data_array.x.values, 
        'y':data_array.y.values 
    }
    
    # Define el orden de las dimensiones
    dims = ('y','x')
    
    # Define los metadatos o atributos
    attrs = {
        'crs': data_array.crs,
        'transform': data_array.transform
    }
    
    data = {}
    for band_index, band_name in positions:
        data[band_name] = xr.DataArray(
            data=data_array.values[band_index], 
            coords=coords, 
            dims=dims, 
            name=band_name, 
            attrs=attrs
        )
    
    return data

tif = load_ideam_geotif(path)
tif

{'green': <xarray.DataArray 'green' (y: 3687, x: 3705)>
 array([[ 275.5,  260. ,  218. , ..., 1076. ,  999. ,  950.5],
        [ 325. ,  251. ,  264.5, ..., 1017. , 1002.5,  859. ],
        [ 278. ,  322. ,  262.5, ...,  961. ,  880.5,  801.5],
        ...,
        [ 562. ,  599. ,  553. , ..., 1085.5, 1022. , 1139. ],
        [ 541. ,  564. ,  568. , ...,  879.5,  782. ,  803. ],
        [ 495. ,  509. ,  566. , ...,  973. ,  869. ,  793. ]], dtype=float32)
 Coordinates:
   * x        (x) float64 -73.0 -73.0 -73.0 -73.0 -73.0 -73.0 -73.0 -73.0 ...
   * y        (y) float64 12.0 12.0 12.0 12.0 12.0 12.0 12.0 12.0 12.0 12.0 ...
 Attributes:
     crs:        +init=epsg:4326
     transform:  (0.0002699949999999989, 0.0, -73.00016812, 0.0, -0.0002713020...,
 'swir2': <xarray.DataArray 'swir2' (y: 3687, x: 3705)>
 array([[ 181. ,  175.5,  155. , ..., 2120.5, 2009. , 1928. ],
        [ 205. ,  168.5,  178.5, ..., 2081. , 2016.5, 1807. ],
        [ 181. ,  185. ,  176. , ..., 2096. , 1915. , 

In [14]:
path = '../data/prueba/pequena.tif'
data = xr.open_rasterio(path)
inv = data[dict(band=slice(0, 6))].values
inv

array([[[ 275.5,  260. ,  218. , ..., 1076. ,  999. ,  950.5],
        [ 325. ,  251. ,  264.5, ..., 1017. , 1002.5,  859. ],
        [ 278. ,  322. ,  262.5, ...,  961. ,  880.5,  801.5],
        ...,
        [ 562. ,  599. ,  553. , ..., 1085.5, 1022. , 1139. ],
        [ 541. ,  564. ,  568. , ...,  879.5,  782. ,  803. ],
        [ 495. ,  509. ,  566. , ...,  973. ,  869. ,  793. ]],

       [[ 181. ,  175.5,  155. , ..., 2120.5, 2009. , 1928. ],
        [ 205. ,  168.5,  178.5, ..., 2081. , 2016.5, 1807. ],
        [ 181. ,  185. ,  176. , ..., 2096. , 1915. , 1777. ],
        ...,
        [ 815. ,  884. ,  815. , ..., 1032. , 1271. , 2090.5],
        [ 807. ,  855. ,  866. , ..., 1507. , 1274. , 1364. ],
        [ 699. ,  782. ,  852. , ..., 1195. , 1026. , 1370. ]],

       [[ 213.5,  196.5,  153. , ..., 2378. , 2290. , 2236. ],
        [ 251. ,  196.5,  194.5, ..., 2366. , 2334.5, 2140. ],
        [ 204. ,  245. ,  195. , ..., 2306. , 2188.5, 2111.5],
        ...,
        [394

In [15]:
def predict(*instance):
    print(instance)
    proba = model.predict([[ instance ]])
    index = np.argmax(proba)
    # classes = ['agua','bosque','agua_o_bosque','ninguno']
    classes = [1,2,3,4]
    return classes[index]

In [16]:
# from dask.distributed import Client, progress
# from distributed.diagnostics import progress

# import dask.array as da
# from time import sleep

# import warnings
# warnings.filterwarnings(action='ignore')

# import logging
# logger = logging.getLogger("distributed.utils_perf")
# logger.setLevel(logging.ERROR)

# from dask.diagnostics import ProgressBar
# ProgressBar().register()

# client = Client(processes=False, threads_per_worker=4,n_workers=1, memory_limit='2GB')
# client

In [17]:
# data = xr.open_rasterio('../data/prueba/pequena.tif',chunks={'band': 6, 'x': 1024, 'y': 1024})
# data.variable.data

In [18]:
# import warnings
# warnings.filterwarnings('once')

# # import logging
# # logger = logging.getLogger("distributed.utils_perf")
# # logger.setLevel(logging.ERROR)

# from dask.diagnostics import ProgressBar
# ProgressBar().register()

# def hola(*instance):
#     print(instance)
#     return 1

# with ProgressBar():
# client = Client(processes=False, threads_per_worker=1,n_workers=1, memory_limit='2GB')
# client
# future = np.apply_along_axis(predict, 0, data)
# future.compute()

In [19]:
# np.apply_along_axis(predict, 0, inv)

In [20]:
# np.apply_along_axis(sum, 0, inv)

In [21]:
# blue	green	red	nir	swir1	swir2
# function = lambda *args,**kwags: print(list(args))
# xr.apply_ufunc(function, tif['blue'].values, tif['green'].values,tif['red'].values,tif['nir'].values,tif['swir1'].values,tif['swir2'].values)

In [22]:
# vpredict([tif['blue'].values, tif['green'].values,tif['red'].values,tif['nir'].values,tif['swir1'].values,tif['swir2'].values])

In [23]:
# sum([tif['blue'].values, tif['green'].values,tif['red'].values,tif['nir'].values,tif['swir1'].values,tif['swir2'].values])

In [24]:
# tif['green'].values

In [25]:
# len(tif)

In [26]:
# tif['red'].plot()

In [27]:
# shape = tif['red'].shape
# result = np.zeros(shape=shape)
# shape

In [28]:
# max_data = 27667864
# max_data = 10000
# len(np.indices((7373, 7408))[0])
# data_points = np.argwhere(~np.isnan(tif['blue'].values))
# data_points
# data_points = random.sample(list(data_points), max_data)
# data_points = list(data_points)[:max_data]

# data_list = 
# def process(i,j):
# for i,j in data_points:
# def compute():
#     shape_i = tif['blue'].shape[0]
#     shape_j = tif['blue'].shape[1]

#     for i in range(shape_i):
#         for j in range(shape_j):
#             blue = tif['blue'].values[i,j]
#             green = tif['green'].values[i,j]
#             red = tif['red'].values[i,j]
#             nir = tif['nir'].values[i,j]
#             swir1 = tif['swir1'].values[i,j]
#             swir2 = tif['swir2'].values[i,j]
#             instance = [blue,green,red,nir,swir1,swir2]
#             result[i,j] = predict_with_class(model,instance)
#     #         result[i,j] = 1

# %time compute()

In [29]:
# show(result)

In [30]:
# result

In [31]:
# from dask.distributed import Client, progress
# import dask.array as da

In [32]:
# client = Client(processes=False, threads_per_worker=4,n_workers=1, memory_limit='20GB')
# client

In [33]:
# x = da.random.random((10000, 10000), chunks=(1000, 1000))
# type(x)

In [34]:
# y = x + x.T
# z = y[::2, 5000:].mean(axis=1)
# z

In [35]:
# y.compute()